In [1]:
from dune_client.client import DuneClient
import json
from dotenv import load_dotenv
import os

# Load and read environment variables from .env file
load_dotenv()
dune_api_key = os.getenv("DUNE_API_KEY")


In [2]:
dune = DuneClient(dune_api_key)
# query_result = dune.get_latest_result(871360)
query_result_df = dune.get_latest_result_dataframe(871360)
query_result_df.to_csv("./Top1000Delegates.csv")


In [11]:
query_result_df.head()

,delegate,delegate_rank,delegate_name,delegator_link,col1,dt_voting_power,pct_voting_power,col2,vp_change_30d,pct_vp_change_30d,col3,num_delegators,pct_delegators,delegate_name_raw,delegate_address,rn,running_pct_voting_power,running_dt_voting_power,num_delegates_to_majority,num_delegates_to_quorum
0,0x3eee61b92c36e97be6319bf9096a1ac3c04a1466,1,"<a href=""https://vote.optimism.io/delegates/0x...",<a href='https://dune.com/oplabspbc/op-token-h...,|,1.018548e+07,0.113756,|,105028.741461,0.01041904802156521,|,1940,0.002556,Anticapture Commission,0x3eee61b92c36e97be6319bf9096a1ac3c04a1466,1,0.113756,1.018548e+07,11,4
1,0xf11b6a8c3cb8bb7dbc1518a613b10ceb0bbfc06b,2,"<a href=""https://vote.optimism.io/delegates/0x...",<a href='https://dune.com/oplabspbc/op-token-h...,|,5.214280e+06,0.058235,|,-912.530933,-0.00017497551230160314,|,665,0.000876,0xf11b6a8c3cb8bb7dbc1518a613b10ceb0bbfc06b,0xf11b6a8c3cb8bb7dbc1518a613b10ceb0bbfc06b,1,0.171991,1.539976e+07,11,4
2,0x1b686ee8e31c5959d9f5bbd8122a58682788eead,3,"<a href=""https://vote.optimism.io/delegates/0x...",<a href='https://dune.com/oplabspbc/op-token-h...,|,5.068114e+06,0.056603,|,153743.346077,0.03128444076368586,|,9776,0.012882,L2BEAT,0x1b686ee8e31c5959d9f5bbd8122a58682788eead,1,0.228594,2.046788e+07,11,4
3,0xeff8d84e0fd304550da242040ccd45bd44ce71f1,4,"<a href=""https://vote.optimism.io/delegates/0x...",<a href='https://dune.com/oplabspbc/op-token-h...,|,5.028728e+06,0.056163,|,387.695743,7.710213535638393e-05,|,556,0.000733,0xeff8d84e0fd304550da242040ccd45bd44ce71f1,0xeff8d84e0fd304550da242040ccd45bd44ce71f1,1,0.284757,2.549660e+07,11,4
4,0x5e349eca2dc61abcd9dd99ce94d04136151a09ee,5,"<a href=""https://vote.optimism.io/delegates/0x...",<a href='https://dune.com/oplabspbc/op-token-h...,|,4.171481e+06,0.046589,|,-128250.958046,-0.029827666364952557,|,81294,0.107119,Linda Xie,0x5e349eca2dc61abcd9dd99ce94d04136151a09ee,1,0.331345,2.966809e+07,11,4


In [ ]:
# import requests

# url = "https://api.dune.com/api/v1/query/{queryId}"

# headers = {"X-DUNE-API-KEY": dune_api_key}

# response = requests.request("GET", url, headers=headers)

# print(response.text)

In [13]:
df_vote_cast = dune.get_latest_result_dataframe(3848309)
df_vote_cast.to_csv("./vote_cast.csv")


In [16]:
df_proposals = dune.get_latest_result_dataframe(3866186)
df_proposals.to_csv("./proposals.csv")